# Visual Odometry (VO)

In this assignment, you do not need. a GPU. You will use the pykitti module and KITTI odometry dataset.

You can download the odometry data from [here](https://drive.google.com/file/d/1PJOUnM3nEwDpqiRvfqUnkNPJZpM4PKYV/view?usp=sharing).

## Monocular VO with OpenCV on KITTI

For each consecutive frame pair in the sequence, you will compute the relative pose between the frames and visualize it. You will use:

* pykitti code similar to what you wrote in mvs part to load the seqeunce with ground-truth info. (Check out the [demo code](https://github.com/utiasSTARS/pykitti/blob/master/demos/demo_odometry.py))
* OpenCV functions to compute and visualize the features and the essential matrix.

Please follow these steps to complete the assignment:

1. You can use the ORB Feature to do the feature matching:
    `orb = cv2.ORB_create()` to create the ORB object
    and then `orb.detectAndCompute()` to find the keypoints and descriptors on both frames

2. You can use brute-force matcher to match ORB descriptors:
    `bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)`

3. After matching the descriptors, sort the matched keypoints.

4. Draw matches on the two images using the `cv2.drawMatches()` function.

5. Compute the essential matrix using the `cv2.findEssentialMat()` function. Note that you need the matching points and the instrinsics for this function. 

6. Extract the rotation and translation from the essential matrix using the `cv2.recoverPose()` function.

7. Multiply the estimated rotation and translation with the previous rotation and translation. Initialize rotation to identity and translation to zeros on the first frame.

8. Display the current image with the keypoints on it using the `cv2.drawKeypoints()` function.

9. Update the previous rotation and translation as the current rotation and translation.

10. Draw the estimated trajectory as blue and ground-truth trajectory as green. You can use the `cv2.circle()` function.


You can create a video of your visualization of images and poses for the provided sequence.

**Bonus**: Compute the absolute trajectory error between the estimated trajectory and the ground-truth trajectory. 

Some examples repositories that might be useful:
* https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/
* https://github.com/uoip/monoVO-python


In [1]:
width = 1241.0
height = 376.0
fx, fy, cx, cy = [718.8560, 718.8560, 607.1928, 185.2157]


In [2]:
import numpy as np
trajMap = np.zeros((1000, 1000, 3), dtype=np.uint8)

In [3]:
os.listdir('./KITTI_odometry/sequences/09/')

['image_2', 'image_3', 'calib.txt', 'times.txt']

In [4]:
import os
path_dir = './KITTI_odometry/sequences/09/image_2'
paths = [path for path in os.listdir(path_dir) if path.endswith('.png')]
paths = sorted(paths)

In [5]:
# load gt trajectory
gt_T = []
with open('./KITTI_odometry/poses/09.txt') as f:
    for line in f:
        arr = list(map(float, line.split(' ')))
        gt_T.append(np.array(arr).reshape(3, 4))

In [6]:
import cv2
prev_img = cv2.imread(os.path.join(path_dir, paths[0]), 0)
prev_R, prev_t = None, None
prev_gt_R, prev_gt_t = None, None
for i, path in enumerate(paths[1:]):
    cur_img = cv2.imread(os.path.join(path_dir, path), 0)
    
    orb = cv2.ORB_create(nfeatures=6000)

    kp1, des1 = orb.detectAndCompute(prev_img, None)
    kp2, des2 = orb.detectAndCompute(cur_img, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)


    img_matching = cv2.drawMatches(cur_img, kp1, prev_img, kp2, matches[0:100], None)

    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

    E, mask = cv2.findEssentialMat(pts1, pts2, focal=fx, pp=(cx, cy), method=cv2.RANSAC, prob=0.999, threshold=1)
    pts1 = pts1[mask.ravel() == 1]
    pts2 = pts2[mask.ravel() == 1]

    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, focal=fx, pp=(cx, cy))


    R = R.transpose()
    t = np.matmul(R, t)

    if prev_R is None:
        curr_R, curr_t = R, t
    else:
        curr_R, curr_t = np.matmul(prev_R, R), np.matmul(prev_R, t) + prev_t

    # draw the current image with keypoints
    curr_img_kp = cv2.drawKeypoints(cur_img, kp2, None, color=(0, 255, 0), flags=0)
    cv2.imshow('keypoints from current image', curr_img_kp)


    # draw estimated trajectory (blue) and gt trajectory (red)
    offset_draw = (int(1000/2))
    cv2.circle(trajMap, (-int(curr_t[0])+offset_draw, int(curr_t[2])+offset_draw), 1, (255,0,0), 2)
    cv2.circle(trajMap, (int(gt_T[i][0, 3])+offset_draw, -int(gt_T[i][2, 3])+offset_draw), 1, (0,0,255), 2)
    cv2.imshow('Trajectory', trajMap)
    cv2.waitKey(1)

    prev_R, prev_t = curr_R, curr_t
    prev_img = cur_img
    
cv2.imwrite('trajMap.png', trajMap)

True